<a href="https://colab.research.google.com/github/ekacs/scrap-inaportnet-22/blob/main2/webscrap_INAPORTNET.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import os
import re
from six.moves import urllib
from unicodedata import normalize
import numpy as np
import io
import datetime
from datetime import datetime, timedelta
from dateutil.tz import tzutc, tzlocal
from google.colab import files


In [ ]:
kode_lokasi='IDJKT'     #diisi kode lokasi pelabuhan
tahun=2022              #diisi tahun
jenis_layanan='RKBM'    #diisi kode layanan inaportnet yg mau discrape, diakhir kode dikasi TOLONG ditambahkan tanda kurung tutup ')'
berapa_bulan=2          #diisi berapa bulan
selisih_waktu_UTC=0     # diisi waktu selisih perbedaan dg waktu UTC



In [ ]:

daftarbulan=['00','01','02','03','04','05','06','07','08','09','10','11','12']
urls=[]
daftarlink=[]   


for i in range(1,berapa_bulan+1):
  
  #FYI gaes syntax format URL = https://monitoring-inaportnet.dephub.go.id/site/port_detail_service?unlocode=<kode_lokasi>&bulan=<angka_bulan_2digit>&tahun=<angka_tahun_4digit>&service_code=<kode_layanan>
  urlpage = ('https://monitoring-inaportnet.dephub.go.id/site/port_detail_service?unlocode={kd_lokasi}&bulan={n}&tahun=2022&service_code={jns_produk}').format(kd_lokasi=kode_lokasi, n=(daftarbulan[i]), jns_produk=jenis_layanan)
  urls.append(urlpage)


for url in urls:
    my_url = requests.get(url) 
  
    soup = bs(my_url.content, 'html.parser')
    #print(soup)
    s = soup('a', class_='btn btn-primary ls-modal')
    #print(s)
    for link in soup.find_all('a', attrs={'href': re.compile("kode_perusahaan=")}):
      daftarlink.append('https://monitoring-inaportnet.dephub.go.id'+link.get('href')+'&download=1')
      #daftarlink.append(link.get('href'))
      #print(daftarlink)

#len(daftarlink)
#print(urls)
#print(urlpage)


hasil_scrape = pd.DataFrame(pd.read_csv(daftarlink[0]))

for i in range(0,len(daftarlink)):
  data = pd.read_html(daftarlink[i])
  df = pd.DataFrame(data[0])

  hasil_scrape = pd.concat([hasil_scrape,df],axis=0)



#hasil_scrape
#hasil_scrape.dropna(subset=["No"])


#PERHITUNGAN SELISIH WAKTU

df=hasil_scrape
df['Waktu Permohonan'] = pd.to_datetime(df['Waktu Permohonan'])
df['Waktu Respon'] = pd.to_datetime(df['Waktu Respon'])

df['waktu_permohonan_real'] = df['Waktu Permohonan']+pd.DateOffset(hours=selisih_waktu_UTC) 
df['selisih_waktu'] = df['Waktu Respon']-df['waktu_permohonan_real']
#hasil_scrape.info()



#cari data anomali Waktu Respon lebih dulu ketimbang Waktu Permohonan (asumsi ada bug di database inaportnet)

hasil_scrape_bug=hasil_scrape[hasil_scrape['selisih_waktu']<timedelta(seconds=0)]
hasil_scrape_ok=hasil_scrape[hasil_scrape['selisih_waktu']>timedelta(seconds=0)]



#make criteria in time difference
def sel(hasil_scrape):
    if hasil_scrape['selisih_waktu'] < timedelta(seconds=31):
        return "Kurang dari 30 detik"
    elif hasil_scrape['selisih_waktu'] < timedelta(minutes=30):
        return "Antara 30 detik - 30 menit"
    elif hasil_scrape['selisih_waktu'] < timedelta(hours=1):
        return "Antara 30 - 60 menit"
    elif hasil_scrape['selisih_waktu'] < timedelta(hours=6):
        return "Antara 1 - 6 jam"
    elif hasil_scrape['selisih_waktu'] < timedelta(hours=12):
        return "Antara 6 - 12 jam"
    elif hasil_scrape['selisih_waktu'] < timedelta(days=1):
        return "Antara 12 - 24 jam"
    else:
        return "Lebih dari 24 jam"

hasil_scrape_ok['lama_pelayanan'] = hasil_scrape_ok.apply(sel, axis=1)
sort_data = hasil_scrape_ok.sort_values("selisih_waktu", axis = 0, ascending = True)


lama_pelayanan = hasil_scrape_ok.groupby('lama_pelayanan')['lama_pelayanan'].count().reset_index(name='jumlah')
lama_pelayanan['persentase'] = lama_pelayanan['jumlah']/len(hasil_scrape_ok)*100
#lama_pelayanan


#gruoping month-day pelayanan

month_mapping={
    1: 'January', 
    2: 'February', 
    3: 'March', 
    4: 'April', 
    5: 'May',
    6: 'June', 
    7: 'July',
    8: 'August'
} 
dw_mapping={
    0: 'Monday', 
    1: 'Tuesday', 
    2: 'Wednesday', 
    3: 'Thursday', 
    4: 'Friday',
    5: 'Saturday', 
    6: 'Sunday'
} 

hasil_scrape_ok['bulan permohonan'] = hasil_scrape_ok['jam_permohonan_UTC].dt.month.map(month_mapping)
hasil_scrape_ok['hari permohonan'] = hasil_scrape_ok['jam_permohonan_UTC'].dt.dayofweek.map(dw_mapping)

bulan_permohonan = hasil_scrape_ok.groupby('bulan permohonan')['bulan permohonan'].count().reset_index(name='jumlah')
bulan_permohonan['persentase'] = bulan_permohonan['jumlah']/len(datagabung)*100

def nomor(bulan_permohonan):
    if bulan_permohonan['bulan permohonan'] == 'January':
        return 1
    elif bulan_permohonan['bulan permohonan'] == 'February':
        return 2
    elif bulan_permohonan['bulan permohonan'] == 'March':
        return 3
    elif bulan_permohonan['bulan permohonan'] == 'April':
        return 4
    elif bulan_permohonan['bulan permohonan'] == 'May':
        return 5
    elif bulan_permohonan['bulan permohonan'] == 'June':
        return 6
    elif bulan_permohonan['bulan permohonan'] == 'July':
        return 7
    else:
        return 8

bulan_permohonan['nomor'] = bulan_permohonan.apply(nomor, axis=1)
bulan_permohonan = bulan_permohonan.sort_values("nomor", axis = 0, ascending = True)
#bulan_permohonan

#stat hari permohnonan

def nomor_hari(hari_permohonan):
    if hari_permohonan['hari permohonan'] == 'Monday':
        return 1
    elif hari_permohonan['hari permohonan'] == 'Tuesday':
        return 2
    elif hari_permohonan['hari permohonan'] == 'Wednesday':
        return 3
    elif hari_permohonan['hari permohonan'] == 'Thursday':
        return 4
    elif hari_permohonan['hari permohonan'] == 'Saturday':
        return 5
    elif hari_permohonan['hari permohonan'] == 'Sunday':
        return 6
    else:
        return 7
    
hari_permohonan = hasil_scrape_ok.groupby('hari permohonan')['hari permohonan'].count().reset_index(name='jumlah')
hari_permohonan['Nomor'] = hari_permohonan.apply(nomor_hari, axis=1)
hari_permohonan = hari_permohonan.sort_values("Nomor", axis = 0, ascending = True)
#hari_permohonan


#pemabgian rentang waktu permohonan
batas_waktu1 = pd.to_datetime('06:00:00', format='%H:%M:%S')
batas_waktu2 = pd.to_datetime('12:00:00', format='%H:%M:%S')
batas_waktu3 = pd.to_datetime('18:00:00', format='%H:%M:%S')

def jam(hasil_scrape_ok):
    if hasil_scrape_ok['jam permohonan'] < batas_waktu1:
        return "Jam 00.00 s.d. 06.00"
    elif hasil_scrape_ok['jam permohonan'] < batas_waktu2:
        return "Jam 06.00 s.d. 12.00"
    elif hasil_scrape_ok['jam permohonan'] < batas_waktu3:
        return "Jam 12.00 s.d. 18.00"
    else:
        return "Jam 18.00 s.d. 24.00"

hasil_scrape_ok['jam permohonan'] = hasil_scrape_ok['jam_permohonan_UTC'].dt.time
hasil_scrape_ok['jam permohonan'] = pd.to_datetime(hasil_scrape_ok['jam permohonan'], format='%H:%M:%S')
hasil_scrape_ok['pembagian jam permohonan'] = hasil_scrape_ok.apply(jam, axis=1)

waktu_permohonan = datagabung_tanpabug.groupby('pembagian jam permohonan')['pembagian jam permohonan'].count().reset_index(name='jumlah')
waktu_permohonan['persentase'] = waktu_permohonan['jumlah']/len(datagabung_tanpabug)*100
#waktu_permohonan


#data hasil scrape yg anomali (asumsi ada bug)

In [ ]:

hasil_scrape_bug.dropna(subset=['No']).to_csv('hasil_scrape_bug.csv')
files.download('hasil_scrape_bug.csv')


#statistik durasi lama pelayanan

In [ ]:
lama_pelayanan

,lama_pelayanan,jumlah,persentase
0,Antara 1 - 6 jam,1,0.050942
1,Antara 6 - 12 jam,1927,98.166072
2,Lebih dari 24 jam,35,1.782985


#statistik waktu pelayanan

#statistik hari pelayanan

#statistik bulan pelayanan

In [ ]:
bulan_permohonan

In [ ]:

for url in urls:
    my_url = requests.get(url) 
  
    soup = bs(my_url.content, 'html.parser')
    #print(soup)
    s = soup('a', class_='btn btn-primary ls-modal')
    #print(s)
    for link in soup.find_all('a', attrs={'href': re.compile("kode_perusahaan=")}):
      daftarlink.append('https://monitoring-inaportnet.dephub.go.id'+link.get('href')+'&download=1')
      #daftarlink.append(link.get('href'))
      #print(daftarlink)

len(daftarlink)
#print(urls)
#print(urlpage)



In [ ]:
hasil_scrape = pd.DataFrame(pd.read_csv(daftarlink[0]))

for i in range(0,len(daftarlink)):
  data = pd.read_html(daftarlink[i])
  df = pd.DataFrame(data[0])

  hasil_scrape = pd.concat([hasil_scrape,df],axis=0)



hasil_scrape
#hasil_scrape.dropna(subset=["No"])


In [ ]:

#cari data anomali Waktu Respon lebih dulu ketimbang Waktu Permohonan (asumsi ada bug di database inaportnet)

hasil_scrape_bug=hasil_scrape[hasil_scrape['selisih_waktu']<timedelta(seconds=0)]
hasil_scrape_ok=hasil_scrape[hasil_scrape['selisih_waktu']>timedelta(seconds=0)]

In [ ]:

#STAT LAMA PELAYANAN
def sel(hasil_scrape):
    if hasil_scrape['selisih_waktu'] < timedelta(seconds=31):
        return "Kurang dari 30 detik"
    elif hasil_scrape['selisih_waktu'] < timedelta(minutes=30):
        return "Antara 30 detik - 30 menit"
    elif hasil_scrape['selisih_waktu'] < timedelta(hours=1):
        return "Antara 30 - 60 menit"
    elif hasil_scrape['selisih_waktu'] < timedelta(hours=6):
        return "Antara 1 - 6 jam"
    elif hasil_scrape['selisih_waktu'] < timedelta(hours=12):
        return "Antara 6 - 12 jam"
    elif hasil_scrape['selisih_waktu'] < timedelta(days=1):
        return "Antara 12 - 24 jam"
    else:
        return "Lebih dari 24 jam"

hasil_scrape_ok['lama_pelayanan'] = hasil_scrape_ok.apply(sel, axis=1)
sort_data = hasil_scrape_ok.sort_values("selisih_waktu", axis = 0, ascending = True)



lama_pelayanan = hasil_scrape_ok.groupby('lama_pelayanan')['lama_pelayanan'].count().reset_index(name='jumlah')
lama_pelayanan['persentase'] = lama_pelayanan['jumlah']/len(hasil_scrape_ok)*100


#STATISTIK WAKTU PERMOHONAN
def jam(datagabung_tanpabug):
    if datagabung_tanpabug['jam permohonan'] < batas_waktu1:
        return "Jam 00.00 s.d. 06.00"
    elif datagabung_tanpabug['jam permohonan'] < batas_waktu2:
        return "Jam 06.00 s.d. 12.00"
    elif datagabung_tanpabug['jam permohonan'] < batas_waktu3:
        return "Jam 12.00 s.d. 18.00"
    else:
        return "Jam 18.00 s.d. 24.00"

datagabung_tanpabug['jam permohonan'] = datagabung_tanpabug['jam_permohonan_UTC'].dt.time
datagabung_tanpabug['jam permohonan'] = pd.to_datetime(datagabung_tanpabug['jam permohonan'], format='%H:%M:%S')
datagabung_tanpabug['pembagian jam permohonan'] = datagabung_tanpabug.apply(jam, axis=1)


In [ ]:
#gruoping month-day pelayanan

month_mapping={
    1: 'January', 
    2: 'February', 
    3: 'March', 
    4: 'April', 
    5: 'May',
    6: 'June', 
    7: 'July',
    8: 'August'
} 
dw_mapping={
    0: 'Monday', 
    1: 'Tuesday', 
    2: 'Wednesday', 
    3: 'Thursday', 
    4: 'Friday',
    5: 'Saturday', 
    6: 'Sunday'
} 

hasil_scrape_ok['bulan permohonan'] = hasil_scrape_ok['jam_permohonan_UTC].dt.month.map(month_mapping)
hasil_scrape_ok['hari permohonan'] = hasil_scrape_ok['jam_permohonan_UTC'].dt.dayofweek.map(dw_mapping)

bulan_permohonan = hasil_scrape_ok.groupby('bulan permohonan')['bulan permohonan'].count().reset_index(name='jumlah')
bulan_permohonan['persentase'] = bulan_permohonan['jumlah']/len(datagabung)*100

def nomor(bulan_permohonan):
    if bulan_permohonan['bulan permohonan'] == 'January':
        return 1
    elif bulan_permohonan['bulan permohonan'] == 'February':
        return 2
    elif bulan_permohonan['bulan permohonan'] == 'March':
        return 3
    elif bulan_permohonan['bulan permohonan'] == 'April':
        return 4
    elif bulan_permohonan['bulan permohonan'] == 'May':
        return 5
    elif bulan_permohonan['bulan permohonan'] == 'June':
        return 6
    elif bulan_permohonan['bulan permohonan'] == 'July':
        return 7
    else:
        return 8

bulan_permohonan['nomor'] = bulan_permohonan.apply(nomor, axis=1)
bulan_permohonan = bulan_permohonan.sort_values("nomor", axis = 0, ascending = True)
bulan_permohonan
